# <center>Curricular Analytics of Vanderbilt University</center>

<center>
    <b>Jonathan Sprinkle</b>
</center>

## 1. Introduction
Install per the instructions of curricular analytics.

In [1]:
using CurricularAnalytics, CurricularVisualization

WebIO._IJuliaInit()

## 2. Load Curricula and Degree Plans


In [2]:
vu_cs = read_csv("./vanderbilt-u-CS.csv")
visualize(vu_cs, notebook=true, scale=1.0)

WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol, Any}(:src => "https://curricula-api-embed.damoursystems.com/", :id => "curriculum", :events => Dict{Symbol, WebIO.JSString}(:load => WebIO.JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":false,\"edit\":false},\"curriculum\":{\"name\":\"Computer Science\",\"dp_name\":\"AY26 BS Degree Plan\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":2,\"target_id\":1,\"type\":\"CurriculumCorequisite\"}],\"name\":\"CHEM 1601\",\"metrics\":{\"centrality\":0,\"complexity\":2.0,\"blocking factor\":0,\"delay factor\":2.0},\"nameSub\":\"General Chemistry\",\"id\":1,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CHEM 1601L\",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"General Chemistry Laboratory\",\"id\":2,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"MATH 1300\",\"metrics\":{\"centrality\":0,\"complexity\":5.0,\"blocking factor\":2,\"delay factor\":3.0},\"nameSub\":\"Accelerated Single Variable Calculus I\",\"id\":3,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":5,\"target_id\":4,\"type\":\"CurriculumCorequisite\"}],\"name\":\"ES 1401\",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\":0,\"delay factor\":3.0},\"nameSub\":\"Introduction to Engineering\",\"id\":4,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":6,\"target_id\":5,\"type\":\"CurriculumCorequisite\"}],\"name\":\"ES 1402\",\"metrics\":{\"centrality\":3,\"complexity\":4.0,\"blocking factor\":1,\"delay factor\":3.0},\"nameSub\":\"Introduction to Engineering\",\"id\":5,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"ES 1403\",\"metrics\":{\"centrality\":0,\"complexity\":5.0,\"blocking factor\":2,\"delay factor\":3.0},\"nameSub\":\"Introduction to Engineering\",\"id\":6,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Open Elective\",\"id\":7,\"credits\":3,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\"CS 1101\",\"metrics\":{\"centrality\":0,\"complexity\":17.0,\"blocking factor\":13,\"delay factor\":4.0},\"nameSub\":\"Programming and Problem Solving\",\"id\":8,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":10,\"target_id\":9,\"type\":\"CurriculumCorequisite\"}],\"name\":\"PHYS 1601\",\"metrics\":{\"centrality\":3,\"complexity\":4.0,\"blocking factor\":1,\"delay factor\":3.0},\"nameSub\":\"General Physics I\",\"id\":9,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"PHYS  1601L\",\"metrics\":{\"centrality\":0,\"complexity\":5.0,\"blocking factor\":2,\"delay factor\":3.0},\"nameSub\":\"General Physics I Laboratory\",\"id\":10,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":3,\"target_id\":11,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"MATH  1301\",\"metrics\":{\"centrality\":3,\"complexity\":4.0,\"blocking factor\":1,\"delay factor\":3.0},\"nameSub\":\"Accelerated Single Variable Calculus II\",\"id\":11,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Liberal Arts Core\",\"id\":12,\"credits\":3,\"nameCanonical\":\"\"}],\"id\":2},{\"name\":\"Term 3\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":9,\"target_id\":13,\"type\":\"CurriculumPrerequisite\"},{\"source_id\":14,\"target_id\":13,\"type\":\"CurriculumCorequisite\"}],\"name\":\"PHYS 1602\",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\"

If you'd like to visualize this curriculum, you can download and use a companion visualization package using (warning this may take some time, and its often easier to download a package using the Julia REPL):

#### Verifying Curricula
In order to determine whether or not a curriculum is valid, use the `is_valid` function, as follows:

In [3]:
errors = IOBuffer()
if is_valid(vu_cs)
    println("Curriculum $(vu_cs.name) is valid")
else
    println("Curriculum $(vu_cs.name) is not valid:")
    print(String(take!(errors)))
end

Curriculum AY26 BS Degree Plan is valid


In order to demonstrate how this function works when a curriculum is not valid, let us introduce a cycle in the curriculum graphs -- which makes the curriculum impossible to satisfy.

## 3. Basic Analytics
Now that we have created the curriculum, associated degree plan, and verified there are no errors, we can quantitatively analyze the plan, and more specifically, the courses. We will first begin with delay factor.

### 3.1 Delay Factor
Many curricula, particularly those in science, technology engineering and math (STEM) fields, contain a set of courses that must be completed in sequential order. It is not uncommon in these programs to find prerequisite pathways consisting of seven or eight courses—they span nearly every term in any possible degree plan. The ability to successfully navigate these long pathways without delay is critical for student success and on-time graduation.

We define the **delay factor** associated with a given course $v_k$ in a curriculum $c$, denoted $d_c(v_k)$, as the number of vertices in the longest path in $G_c$ that passes through $v_k$ (Slim, 2016). 

$G_c$ is known as the *curriculum graph*, where each vertex $v_1, . . . , v_n ∈ V$ represents a requirement (i.e., course) in a curriculum $c$. There is a directed edge $(v_i,v_j) ∈ E$ from requirement $v_i$ to $v_j$ if $v_i$ that must be satisfied prior to the satisfaction of $v_j$.

\begin{equation}
d_c(v_k) = \max_{i,j,k,l} \{ \#(v_i\rightsquigarrow v_k \rightsquigarrow v_j)\}
\end{equation}

We define the delay factor associated with an entire curriculum $c$ as:

\begin{equation}
d(G_c)= \sum_{v_k ∈ V} d_c(v_k)
\end{equation}

### 3.2 Blocking Factor
Another structural factor arises when one course serves as the gateway to many other courses in the curriculum. In this case, if a student is unable to pass the gateway course, they are **blocked** from attempting many of the other courses in the curriculum.

In our example, *Calculus 1* is a foundational first-term course that must be completed before taking  the other major-specific classes in subsequent terms leading to our end goal of completing *Circuits One*. It is obvious that a course which is a prerequisite for a large number of other courses in a curriculum is a highly important course in that curriculum.

We will denote the situation where course $v_j$ is reachable from course $v_i$, via any prerequisite pathway, using $vi\rightsquigarrow v_j$, and $v_i \nrightarrow v_j$ will be used if course $v_j$ is not reachable from course $v_i$. The blocking factor associated with course $v_i$ in curriculum $G_c = (V, E)$, denoted $b_c(v_i)$, is then given by (Slim, 2016):

\begin{equation}
b_c(v_i)= \sum_{v_j ∈ V} I(v_i,v_j)
\end{equation}

where $I$ is the indicator function :

\begin{equation}
= I \begin{cases}
1 & if \space \space v_i\rightsquigarrow v_j\\
0 & if \space \space v_i \nrightarrow v_j
\end{cases}
\end{equation}

We define the blocking factor associated with an entire curriculum $c$ as:

\begin{equation}
b(G_c)= \sum_{v_i ∈ V} b_c(v_i)
\end{equation}

### 3.3 Complexity
After computing blocking and delay factor, a unitless measure for structural complexity can be applied to any curriculum.

We keep in mind that the experimental design explicitly relates this measure to the likelihood that a student can complete a curriculum.

In order to achieve this overall *Complexity* metric, we simply add the blocking and delay factors of the entire curricula:

\begin{equation}
Complexity = b(G_c) + d(G_c)
\end{equation}

### 3.4 Back To Our Example
In our example, all these metrics can be computed using the following commands:

In [5]:
    println("Delay factor = $(delay_factor(vu_cs.curriculum))")
    println("Blocking factor = $(blocking_factor(vu_cs.curriculum))")
    println("Curricular complexity = $(complexity(vu_cs.curriculum))")

Delay factor = (108.0, [2.0, 2.0, 3.0, 3.0, 3.0, 3.0, 1.0, 4.0, 3.0, 3.0, 3.0, 1.0, 3.0, 2.0, 4.0, 3.0, 1.0, 1.0, 3.0, 4.0, 3.0, 4.0, 1.0, 1.0, 1.0, 4.0, 1.0, 1.0, 1.0, 1.0, 3.0, 1.0, 4.0, 1.0, 4.0, 4.0, 4.0, 4.0, 1.0, 1.0, 4.0, 4.0, 1.0, 1.0, 1.0])
Blocking factor = (45, [0, 1, 2, 0, 1, 2, 0, 13, 1, 2, 1, 0, 0, 1, 10, 1, 0, 0, 0, 1, 2, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Curricular complexity = (153.0, Number[2.0, 3.0, 5.0, 3.0, 4.0, 5.0, 1.0, 17.0, 4.0, 5.0, 4.0, 1.0, 3.0, 3.0, 14.0, 4.0, 1.0, 1.0, 3.0, 5.0, 5.0, 11.0, 1.0, 1.0, 1.0, 4.0, 1.0, 1.0, 1.0, 1.0, 3.0, 1.0, 4.0, 1.0, 4.0, 4.0, 4.0, 4.0, 1.0, 1.0, 4.0, 4.0, 1.0, 1.0, 1.0])


## Basic Metrics
If more masic metrics are desired such as:

1. Total credit hours
2. Avg. credits per term
3. Number of terms
4. Max. credits in a term
5. Min. credit term
6. Credit hour variance
7. Max. credit term

The following command may be run. Remember that the variable inside of () is whatever you set your degree plan to equal, in our case dp:

In [6]:
basic_metrics(vu_cs)
vu_cs.metrics

Dict{String, Any} with 8 entries:
  "total credit hours"         => 122
  "avg. credits per term"      => 15.25
  "min. credits in a term"     => 14
  "term credit hour std. dev." => 1.29904
  "number of terms"            => 8
  "max. credits in a term"     => 18
  "min. credit term"           => 1
  "max. credit term"           => 7

# Conclusion 
By treating a curriculum as a formal system that exists within the larger university ecosystem, we have highlighted the fact that it can be directly and rigorously analyzed. An analytical approach to the study of curricula supports not only the ability to make predictions about how curricular changes will effect student progress, but also predictions around the likely impact of particular student success interventions on curricular progression.

We hope that through the use of our Curricular Analytics toolbox administrators in higher education can make changes to their curricula that ultimately improve student success outcomes. 

# References

Heileman, G. L., Abdallah, C.T., Slim, A., and Hickman, M. (2018). Curricular analytics: A framework for quantifying the impact of curricular reforms and pedagogical innovations. www.arXiv.org, arXiv:1811.09676 [cs.CY].

Slim, A. (2016). Curricular Analytics in Higher Education. PhD thesis, University of New Mexico,
Albuquerque, NM.